In [6]:
import rospy
import tf
from kuka_arm.srv import *
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from geometry_msgs.msg import Pose
from mpmath import *
from sympy import *
import numpy
import math

import collections
import types
import traceback
import pprint
pp = pprint.PrettyPrinter(indent=4)

#import sys
#sys.path.append("./kuka_arm/scripts/")

import IK_utils as iku
import IK_server as ik
#from IK_server import *
%load_ext autoreload
#%autoreload # Reload all modules now
%autoreload 1 # Reload all modules imported with %aimport every time before executing the Python code typed.
#%autoreload 2 # Reload all modules (except those excluded by %aimport) every time before executing the Python code typed.
%aimport IK_server
%aimport IK_utils
#List modules on the auto list
%aimport 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Modules to reload:
IK_server IK_utils

Modules to skip:



In [7]:
%%HTML
<style> code {background-color : orange !important;} </style>
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [8]:

dictTestCases = {
    "JAllZero":[[[2.153, 0.0, 1.946], [0, 0, 0, 1.0]], # Gripper link (aka End Effector/EE) Requested Pose [Position, OrientationQuat]
                  [1.85, 0, 1.95], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]],  # InverseKinematic response [WC/Joint5 position, Joint1:6 angles]

    "2_-0p5":[[[1.359, 0.0, 2.664], [0, -0.2472, 0, 0.96891]], # Gripper link (aka End Effector/EE) Requested Pose [Position, OrientationQuat]
                  [1.093, 0, 2.519], [0.0, -0.5, 0.0, 0.0, 0.0, 0.0]],  # InverseKinematic response [WC/Joint5 position, Joint1:6 angles]

    "3_-1p0":[[[1.3696, 0.0, 3.488], [0, -0.47942, 0, 0.87758]], # Gripper link (aka End Effector/EE) Requested Pose [Position, OrientationQuat]
                  [1.2059, 0, 3.233], [0.0, 0.0, -1.0, 0.0, 0.0, 0.0]],  # InverseKinematic response [WC/Joint5 position, Joint1:6 angles]

    1:[[[2.16135,-1.42635,1.55109], [0.708611,0.186356,-0.157931,0.661967]],
                  [1.89451,-1.44302,1.69366], [-0.65,0.45,-0.36,0.95,0.79,0.49]],
              
              
    2:[[[-0.56754,0.93663,3.0038], [0.62073, 0.48318,0.38759,0.480629]],
                  [-0.638,0.64198,2.9988], [-0.79,-0.11,-2.33,1.94,1.14,-3.68]],
              
              
    3:[[[-1.3863,0.02074,0.90986], [0.01735,-0.2179,0.9025,0.371016]],
                  [-1.1669,-0.17989,0.85137], [-2.99,-0.12,0.94,4.06,1.29,-4.12]],             
               }


In [9]:
%autoreload
def RunSingleTestCase(testCase):
    jointAnglesExpRad, wristCenterExp, poseReqRaw = iku.ExtractRawTestValues(testCase)
    poseReq = iku.ConvertRawToPose(poseReqRaw)
    if (ik.g_debugPrint):
        print("@@@@@@@@@@@@@@@@@@@@@@@@ debugPrint @@@@@@@@@@@@@@@@@@@@@@@@")
        print("JAExpRad", jointAngleExpRad)
    
    
    req = type("IKReqest", (object,), {})
    req.poses = [poseReq]
        
    if (True):
        print("===================== ik ==========================")
        wristCenterCalcList, jointAnglesCalcRad = ik.IKCalculateJointAnglesFromPose(poseReq, ik.g_tf, ik.dh, ik.g_dictDHVals) # HERE IT IS FINALLY
        poseCalcUsingExpJA = ik.FKCalcPosefromJointAnglesArr(jointAnglesExpRad)
        poseCalcUsingCalcJA = ik.FKCalcPosefromJointAnglesArr(jointAnglesCalcRad)
        

    if (False):
        print("@@@@@@@@@@@@@@@@@@@@@@@@ debugPrint @@@@@@@@@@@@@@@@@@@@@@@@")
        print("JACalc[0] ", jointAnglesCalcRad[0])
        print("JACalc ", jointAnglesCalcRad)
    
    if (False):
        iku.FKCalcAllPosesfromJointAnglesArr(jointAnglesExpRad)
        
    print("======== RequestedPose:")
    print(poseReq)
    iku.PrintCompare("wristCenter", wristCenterCalcList, wristCenterExp)
    iku.PrintCompare("jointAnglesRad", jointAnglesCalcRad, jointAnglesExpRad)
    iku.PrintCompare("jointAnglesDeg", map(degrees,jointAnglesCalcRad), map(degrees, jointAnglesExpRad))
    iku.PrintCompare("Confirm Gripper FKPositions(expectedJointAngles)", iku.ConvertPoseToRawPosition(poseCalcUsingExpJA), iku.ConvertPoseToRawPosition(poseReq))
    iku.PrintCompare("Confirm Gripper FKQuaternions(expectedJointAngles)", iku.ConvertPoseToRawQuaternion(poseCalcUsingExpJA), iku.ConvertPoseToRawQuaternion(poseReq))
    iku.PrintCompare("Confirm Gripper FKPositions(calcJointAngles)", iku.ConvertPoseToRawPosition(poseCalcUsingCalcJA), iku.ConvertPoseToRawPosition(poseReq))
    iku.PrintCompare("Confirm Gripper FKQuaternions(calcJointAngles)", iku.ConvertPoseToRawQuaternion(poseCalcUsingCalcJA), iku.ConvertPoseToRawQuaternion(poseReq))
    return()

Started individual transform processing...
Finished.
saving tfFile /home/cl/catkin_ws/src/RoboND-Kinematics-Project/kuka_arm/scripts/TransformContainer.pypickle


In [10]:
%autoreload
testCase = dictTestCases["3_-1p0"]
#testCase = dictTestCases["JAllZero"]
testCase = dictTestCases["2_-0p5"]
#testCase = dictTestCases[1]
#testCase = dictTestCases[2]
#testCase = dictTestCases[3]


RunSingleTestCase(testCase)

===================== ik ==========================
======== RequestedPose:
position: 
  x: 1.359
  y: 0.0
  z: 2.664
orientation: 
  x: 0
  y: -0.2472
  z: 0
  w: 0.96891
========= wristCenter calc/expect/err:
['  +1.0930', '  +0.0000', '  +2.5188']
['  +1.0930', '  +0.0000', '  +2.5190']
['  -0.0000', '  +0.0000', '  +0.0002']

========= jointAnglesRad calc/expect/err:
['  +0.0000', '  -0.4999', '  -0.0001', '  +0.0000', '  +0.0004', '  +0.0000']
['  +0.0000', '  -0.5000', '  +0.0000', '  +0.0000', '  +0.0000', '  +0.0000']
['  +0.0000', '  -0.0001', '  +0.0001', '  +0.0000', '  -0.0004', '  -0.0000']

========= jointAnglesDeg calc/expect/err:
['  +0.0000', ' -28.6434', '  -0.0073', '  +0.0000', '  +0.0253', '  +0.0000']
['  +0.0000', ' -28.6479', '  +0.0000', '  +0.0000', '  +0.0000', '  +0.0000']
['  +0.0000', '  -0.0045', '  +0.0073', '  +0.0000', '  -0.0253', '  -0.0000']

========= Confirm Gripper FKPositions(expectedJointAngles) calc/expect/err:
['  +1.3589', '  +2.0000', '  +2

()

In [ ]:
%autoreload
def RunAllTestCases(dictTestCases):
    for keyTestIndex, testCase in dictTestCases.iteritems():
        print("############################ Running test case {0} #####################################".format(keyTestIndex))
        RunSingleTestCase(testCase)
RunAllTestCases(dictTestCases)